# 引入库

In [2]:
import keras
import os
import json
import shutil
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
from mylibs.ProcessBar import ShowProcess
from mylibs import funs
from mylibs.my_contrib import *
keras.__version__

Using TensorFlow backend.


'2.2.4'

# 猫狗分类实验
## 实验目的
创建一个简单图像二分类的卷积模型,掌握以下知识点：
* 数据生成器的使用方法
    ./MyDL/Tensorflow/demo/python/demo_ImageDataGenerator2.py
* 卷积神经网络模型创建：卷积层，密度层，激活函数，损失函数，优化函数
* 模型训练方法
* 模型预测方法
* plt基本使用方法
* 训练曲线绘制
* 可视化FeatureMap
* 可视化神经网络的过滤器
* 可视化类激活的热力图

本实验的数据将作为基础参考

## 实验数据说明
  * 网络数据下载地址：https://www.kaggle.com/c/dogs-vs-cats/data
  * 本地数据存储路径：~/e/dataset_tiptical/cats_and_dogs
  * 实验数据根目录：~/data/cats_and_dogs
      - ./ori      ------------猫狗分类原始数据目录
      - ./lab_base ------------实验方案目录
      - ./lab_base/train ------训练目录
      - ./lab_base/valid ------校验目录
      - ./lab_base/test  ------测试目录
      - ./lab_base/test.jpg ---测试图片
      

## 参考资料
visualization of filters keras 基于Keras的卷积神经网络（CNN）可视化

http://www.cnblogs.com/bnuvincent/p/9612686.html

python深度学习{eep learning with python中文版.pdf}源码

https://github.com/fchollet/deep-learning-with-python-notebooks

数据下载：

https://www.kaggle.com/c/dogs-vs-cats/data

本地数据

~/e/dataset_tiptical/cats_and_dogs


# 实验4
VGG19预训练模型提取特征

## 实验参数

In [3]:
##实验参数
print('\n==============================================')
print('设置实验参数')
lab_name='猫狗分类实验-VGG16预训练模型调参'              #实验名称
data_path='%s/data/cats_and_dogs'%(os.getenv('HOME')) #猫狗分类数据根目录
ori_path='%s/ori'%(data_path)                         #猫狗分类原始文件目录
lab_path='%s/lab_vgg19_fine_tuning'%(data_path)          #实验方案目录
split_num="10000,2000,2000"                           #实验数据分割方案,<1：比例分割，>1：数量分割
batch_size=32                                         #批量大小
data_enhance=False                                    #ImageDataGenerator数据启用数据增强
epochs=10                                             #训练轮次
img_width=224                                         #训练图像宽度
img_height=224                                        #训练图像高度 
test_img_path='%s/test.jpg'%(data_path)               #测试图片路径
images_per_row=16       #图像显示每行显示的单元个数
#feature_map_top_num=12  #FeatureMap前面N层{include_top=False}
img_margin=3            #图像单元空隙
layers_name=['conv2d_1','conv2d_2','conv2d_3','conv2d_4'] #卷积层名称
#layers_name=['conv2d_1'] #卷积层名称
last_conv_layer_name='conv2d_4' #最后一层卷积层
gen_pat_steps=40                           #构造迭代次数
cp_file='%s/checkpoint.h5'%(lab_path)      #ModelCheckpoint 文件路径
his_file='%s/history.json'%(lab_path)      #训练日志文件路径
class_mode='binary'                        #分类方法,'binary':二分类，'categorical':多分类
loss='binary_crossentropy'  #损失函数,'binary_crossentropy':二分类，'categorical_crossentropy':多分类

test_cat_path='%s/test_cat.jpg'%(data_path) #猫的测试图像
test_dog_path='%s/test_dog.jpg'%(data_path) #狗的测试图像


设置实验参数


## 加载数据

In [4]:
##加载数据
print('\n==============================================')
print('加载数据......')
#删除lab_path
#shutil.rmtree(lab_path) if os.path.exists(lab_path) else ''

#数据生成器
(train_gen,valid_gen,test_gen)=DataGen(ori_path,lab_path,reset=True,split_num=split_num
                                   ,img_width=img_width,img_height=img_height
                                   ,batch_size=batch_size,enhance=data_enhance,class_mode=class_mode)


加载数据......
enhance:False
img_height:224
split_num:10000,2000,2000
reset:True
img_width:224
class_mode:binary
batch_size:32
delete folder:/home/hjw/data/cats_and_dogs/lab_vgg19_fine_tuning
imgages_split:/home/hjw/data/cats_and_dogs/ori=>/home/hjw/data/cats_and_dogs/lab_vgg19_fine_tuning
Found 20000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## 创建网络

In [5]:
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import Xception
from keras.applications import inception_v3
from keras.applications import MobileNet

In [10]:
print('构建网络')
## 创建网络
def CreateModel(BaseNet_name,BaseNet=VGG16,trainable_name='block5_conv1',input_shape=(150,150,3),flag=0):
    #print('\n==============================================')
    #print('BaseNet:%s'%(BaseNet_name))
    conv_base=BaseNet(weights='imagenet',include_top=False,input_shape=input_shape)
    if flag==1:
        for layer in conv_base.layers:
            layer.trainable=False
    
    elif flag==2:
        set_trainable = False
        for layer in conv_base.layers:
            if layer.name == trainable_name:
                set_trainable = True
            if set_trainable:
                layer.trainable = True
            else:
                layer.trainable = False
    #conv_base.summary()

    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    #打印模型
    #model.summary()
    #模型编译
    model.compile(loss=loss,
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
    return (conv_base,model)

    
#记录日志回调函数    
class train_callback(keras.callbacks.Callback):
    def __init__(self,log_file,history={},verbose=0):
        super(train_callback,self).__init__() #调用父类构造函数
        self.log_file=log_file #训练日志文件路径
        self.history=history   #训练日志
        self.verbose=verbose   #是否显示保存信息
        
    #on_epoch_end: 在每个epoch结束时调用
    def on_epoch_end(self,epoch,logs=None):
        #最佳日志
        if len(self.history)==0:
            for k,v in logs.items():
                self.history[k]=[v]
        else:
            for k,v in logs.items():
                self.history[k].append(v)
        #保存日志
        json.dump(self.history,open(self.log_file,'w'))
        if self.verbose==1:
            print('更新训练日志(%d):%s'%(len(self.history),self.log_file))  

def TrainModel(model):
    #回调函数保存训练日志    
    his_cb=train_callback(his_file,history=history2)

    #断点训练:monitor监控参数可以通过self.score = self.model.evaluate(self.x_test, self.y_test, verbose=0)的score查询
    checkpoint_cb = ModelCheckpoint(cp_file, monitor='val_acc', verbose=1, save_best_only=True, mode='auto',period=2)
    #EarlyStopping
    earlyStopping_cb=keras.callbacks.EarlyStopping(monitor='acc', patience=3, verbose=0, mode='max')
    #TensorBoard
    #tensorBoard_cb=TensorBoard(log_dir=self.log_dir)
    #回调函数序列
    callbacks_list = [checkpoint_cb,earlyStopping_cb,his_cb]

    history = model.fit_generator(
      train_gen,
      steps_per_epoch=np.ceil(train_gen.samples/batch_size),
      epochs=epochs,
      validation_data=valid_gen,
      validation_steps=50,
      callbacks=callbacks_list)            
    

#简报
def Report(BaseNet_name,model):
    core_valid = model.evaluate_generator(valid_gen, steps=100, max_q_size=10, workers=1, pickle_safe=False,verbose=1)
    preds=model.predict_generator(
        test_gen, 
        steps=None, #预测轮数
        max_queue_size=32, 
        workers=1, 
        use_multiprocessing=False,     
        verbose=1)
    
    preds_acc_val=preds_acc(preds,test_gen)
    print('\n\n')
    print('================测试简报(%s[%s])=================='%(lab_name,lab_path))
    #网络模型
    print('ResNet50')
    print(['%s:%s'%(layer.name,'True' if layer.trainable else 'False') for layer in conv_base.layers])
    conv_base.summary()
    model.summary()
    #网络输入
    print('input_shape:%s'%(model.input.shape))
    #网络训练
    print('train=>epochs :%d,samples:%d,loss:%f,acc:%f'
              %(epochs,len(train_gen.filenames),history.history['loss'][-1],history.history['acc'][-1]))
    #网络评估
    print('valid=>samples:%d,val_loss:%f,val_acc:%f'%(len(valid_gen.filenames),score_valid[0],score_valid[1]))
    #网络测试
    print('test =>samples:%d,acc:%f'%(len(test_gen.filenames),preds_acc_val))

构建网络


## 批量训练

In [11]:
#预训练模型
conv_bases={
    'VGG16':[VGG16,'block_conv5',(150,150,3)],
    'VGG19':[VGG19,'block_conv5',(150,150,3)],
    'ResNet50':[ResNet50,'block_conv5',(224,224,3)],
    'Xception':[Xception,'block_conv5',(224,224,3)],
    'InceptionV3':[inception_v3,'block_conv5',(224,224,3)],
    'MobileNet':[MobileNet,'block_conv5',(224,224,3)]
    }

for k,v in conv_bases.items():
    #def CreateModel(BaseNet_name,BaseNet,input_shape,flag):
    for flag in range(3):
        print('==============%s==============='%(k))
        conv_base,model=CreateModel(k,v[0],v[1],v[2],flag)
        conv_base.summary()
        model.summary()

==============VGG19===============
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 3

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

TypeError: 'module' object is not callable